# Gradient Descent & Optimization Algorithms

Understanding optimization from first principles.

**Goal**: Know how neural networks actually learn - the mechanics of updating weights.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
np.random.seed(42)

---
## Part 1: Vanilla Gradient Descent

The fundamental algorithm: follow the negative gradient downhill.

**Your notes here:**


In [ ]:
# Simple 2D function to minimize: f(x, y) = x^2 + y^2
# Global minimum at (0, 0)

def simple_bowl(x, y):
    """Simple convex function - easy to optimize"""
    return x**2 + y**2

def simple_bowl_grad(x, y):
    """Gradient of simple_bowl"""
    return np.array([2*x, 2*y])

# Visualize the function
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = simple_bowl(X, Y)

fig = plt.figure(figsize=(14, 5))

# 3D surface
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.8)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('f(x, y)')
ax1.set_title('3D Surface')

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contour(X, Y, Z, levels=20)
ax2.clabel(contour, inline=True, fontsize=8)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Contour Plot')
ax2.plot(0, 0, 'r*', markersize=15, label='Global minimum')
ax2.legend()

plt.tight_layout()
plt.show()

### Vanilla Gradient Descent

Update rule:
```
θ_new = θ_old - learning_rate * ∇f(θ_old)
```

**Your notes here:**


In [ ]:
def vanilla_gradient_descent(grad_func, start_pos, lr=0.1, n_iters=50):
    """
    Vanilla GD: just follow the negative gradient.
    
    Args:
        grad_func: function that returns gradient at a point
        start_pos: initial position (numpy array)
        lr: learning rate
        n_iters: number of iterations
    
    Returns:
        trajectory: list of positions visited
    """
    pos = start_pos.copy()
    trajectory = [pos.copy()]
    
    for i in range(n_iters):
        grad = grad_func(pos[0], pos[1])
        pos = pos - lr * grad
        trajectory.append(pos.copy())
    
    return np.array(trajectory)

# Run vanilla GD from point (4, 4)
start = np.array([4.0, 4.0])
traj = vanilla_gradient_descent(simple_bowl_grad, start, lr=0.1, n_iters=20)

# Visualize the path
plt.figure(figsize=(8, 8))
contour = plt.contour(X, Y, Z, levels=20, alpha=0.5)
plt.plot(traj[:, 0], traj[:, 1], 'ro-', linewidth=2, markersize=8, label='GD path')
plt.plot(traj[0, 0], traj[0, 1], 'go', markersize=12, label='Start')
plt.plot(0, 0, 'r*', markersize=15, label='Global minimum')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Vanilla Gradient Descent Path')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Started at: {traj[0]}")
print(f"Ended at: {traj[-1]}")
print(f"Distance from minimum: {np.linalg.norm(traj[-1]):.6f}")

### Learning Rate Sensitivity

Learning rate is the most important hyperparameter.

**Your notes here:**


In [ ]:
# Compare different learning rates
learning_rates = [0.05, 0.3, 0.9, 1.1]

fig, axes = plt.subplots(2, 2, figsize=(12, 12))

for ax, lr in zip(axes.flat, learning_rates):
    try:
        traj = vanilla_gradient_descent(simple_bowl_grad, start, lr=lr, n_iters=20)
        
        ax.contour(X, Y, Z, levels=20, alpha=0.5)
        ax.plot(traj[:, 0], traj[:, 1], 'ro-', linewidth=2, markersize=6)
        ax.plot(traj[0, 0], traj[0, 1], 'go', markersize=10)
        ax.plot(0, 0, 'r*', markersize=15)
        ax.set_xlim(-5, 5)
        ax.set_ylim(-5, 5)
        ax.set_title(f'Learning Rate = {lr}')
        ax.grid(True, alpha=0.3)
        
        if lr >= 1.0:
            ax.text(0, 4.5, 'DIVERGES!', fontsize=14, color='red', 
                   ha='center', weight='bold')
    except:
        ax.text(0.5, 0.5, f'LR={lr}\nDIVERGED', transform=ax.transAxes,
               fontsize=16, ha='center', va='center')

plt.tight_layout()
plt.show()

print("Too small: slow convergence")
print("Too large: oscillates or diverges")
print("Just right: smooth, fast convergence")

---
## Part 2: Batch vs Stochastic vs Mini-Batch GD

**Your notes here:**


In [ ]:
# Generate a simple linear regression dataset
def generate_regression_data(n_samples=100):
    X = np.random.randn(n_samples, 1) * 2
    y = 3 * X.squeeze() + 7 + np.random.randn(n_samples) * 0.5
    return X, y

X_train, y_train = generate_regression_data(100)

plt.figure(figsize=(8, 6))
plt.scatter(X_train, y_train, alpha=0.6)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Regression Data: y = 3x + 7 + noise')
plt.grid(True)
plt.show()

In [ ]:
def mse_loss(y_true, y_pred):
    """Mean squared error"""
    return np.mean((y_true - y_pred)**2)

def compute_gradient(X, y, w, b):
    """
    Gradient of MSE loss for linear regression.
    
    y_pred = X @ w + b
    loss = mean((y - y_pred)^2)
    
    dL/dw = -2 * mean(X * (y - y_pred))
    dL/db = -2 * mean(y - y_pred)
    """
    n = len(y)
    y_pred = X @ w + b
    error = y - y_pred.squeeze()
    
    dw = -2 * np.mean(X * error.reshape(-1, 1), axis=0)
    db = -2 * np.mean(error)
    
    return dw, db

### Batch Gradient Descent

Uses ALL data to compute gradient every step.

**Your notes here:**


In [ ]:
def batch_gd(X, y, lr=0.01, n_epochs=100):
    """
    Batch GD: compute gradient using ALL samples every iteration.
    
    + Smooth, stable updates
    - Slow for large datasets (must process all data for one update)
    """
    w = np.random.randn(1)
    b = 0.0
    losses = []
    
    for epoch in range(n_epochs):
        # Compute gradient using ALL data
        dw, db = compute_gradient(X, y, w, b)
        
        # Update
        w -= lr * dw
        b -= lr * db
        
        # Track loss
        y_pred = X @ w + b
        loss = mse_loss(y, y_pred.squeeze())
        losses.append(loss)
    
    return w, b, losses

w_batch, b_batch, losses_batch = batch_gd(X_train, y_train, lr=0.1, n_epochs=100)

print(f"True parameters: w=3, b=7")
print(f"Learned (Batch GD): w={w_batch[0]:.3f}, b={b_batch:.3f}")

### Stochastic Gradient Descent (SGD)

Uses ONE random sample to compute gradient.

**Your notes here:**


In [ ]:
def stochastic_gd(X, y, lr=0.01, n_epochs=100):
    """
    Stochastic GD: compute gradient using ONE random sample.
    
    + Can escape local minima (noisy updates)
    + Faster iterations
    - Very noisy, can be unstable
    """
    w = np.random.randn(1)
    b = 0.0
    losses = []
    
    n = len(y)
    
    for epoch in range(n_epochs):
        # Shuffle data each epoch
        indices = np.random.permutation(n)
        
        for i in indices:
            # Compute gradient using SINGLE sample
            Xi = X[i:i+1]
            yi = y[i:i+1]
            dw, db = compute_gradient(Xi, yi, w, b)
            
            # Update
            w -= lr * dw
            b -= lr * db
        
        # Track loss on full dataset
        y_pred = X @ w + b
        loss = mse_loss(y, y_pred.squeeze())
        losses.append(loss)
    
    return w, b, losses

w_sgd, b_sgd, losses_sgd = stochastic_gd(X_train, y_train, lr=0.01, n_epochs=100)

print(f"True parameters: w=3, b=7")
print(f"Learned (SGD): w={w_sgd[0]:.3f}, b={b_sgd:.3f}")

### Mini-Batch Gradient Descent

Best of both worlds: use a small batch of samples.

**Your notes here:**


In [ ]:
def minibatch_gd(X, y, lr=0.01, n_epochs=100, batch_size=16):
    """
    Mini-batch GD: compute gradient using a BATCH of samples.
    
    + Balance between stability and speed
    + Efficient on GPUs (parallel computation)
    + Default in practice (batch_size typically 32-256)
    """
    w = np.random.randn(1)
    b = 0.0
    losses = []
    
    n = len(y)
    
    for epoch in range(n_epochs):
        # Shuffle data
        indices = np.random.permutation(n)
        
        # Process in mini-batches
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            batch_indices = indices[start:end]
            
            # Compute gradient on mini-batch
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            dw, db = compute_gradient(X_batch, y_batch, w, b)
            
            # Update
            w -= lr * dw
            b -= lr * db
        
        # Track loss
        y_pred = X @ w + b
        loss = mse_loss(y, y_pred.squeeze())
        losses.append(loss)
    
    return w, b, losses

w_mini, b_mini, losses_mini = minibatch_gd(X_train, y_train, lr=0.1, 
                                            n_epochs=100, batch_size=16)

print(f"True parameters: w=3, b=7")
print(f"Learned (Mini-batch): w={w_mini[0]:.3f}, b={b_mini:.3f}")

In [ ]:
# Compare all three
plt.figure(figsize=(10, 6))
plt.plot(losses_batch, label='Batch GD', linewidth=2)
plt.plot(losses_sgd, label='SGD', alpha=0.7)
plt.plot(losses_mini, label='Mini-batch GD', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Batch vs SGD vs Mini-Batch Gradient Descent')
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.show()

print("Notice:")
print("- Batch GD: smooth but may converge slowly")
print("- SGD: noisy but can escape bad regions")
print("- Mini-batch: best trade-off (used in practice)")

---
## Part 3: Momentum

Momentum accelerates GD by accumulating velocity.

**Analogy**: Ball rolling down a hill builds up speed.

**Your notes here:**


In [ ]:
def momentum_gd(grad_func, start_pos, lr=0.01, momentum=0.9, n_iters=50):
    """
    Momentum GD: accumulate velocity.
    
    v_t = β * v_{t-1} + ∇f(θ)
    θ_t = θ_{t-1} - lr * v_t
    
    β (momentum coefficient): typically 0.9
    - Higher β = more "inertia", smoother path
    - Lower β = less memory of past gradients
    """
    pos = start_pos.copy()
    velocity = np.zeros_like(pos)
    trajectory = [pos.copy()]
    
    for i in range(n_iters):
        grad = grad_func(pos[0], pos[1])
        
        # Update velocity (accumulate past gradients)
        velocity = momentum * velocity + grad
        
        # Update position using velocity
        pos = pos - lr * velocity
        trajectory.append(pos.copy())
    
    return np.array(trajectory)

In [ ]:
# Compare vanilla GD vs Momentum on a harder function
def ravine_func(x, y):
    """Ravine function - steep in one direction, shallow in another"""
    return x**2 + 10*y**2

def ravine_grad(x, y):
    return np.array([2*x, 20*y])

# Visualize ravine
x = np.linspace(-10, 10, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = ravine_func(X, Y)

start = np.array([8.0, 3.0])

traj_vanilla = vanilla_gradient_descent(ravine_grad, start, lr=0.05, n_iters=50)
traj_momentum = momentum_gd(ravine_grad, start, lr=0.05, momentum=0.9, n_iters=50)

plt.figure(figsize=(12, 6))

plt.subplot(121)
plt.contour(X, Y, Z, levels=30, alpha=0.5)
plt.plot(traj_vanilla[:, 0], traj_vanilla[:, 1], 'ro-', label='Vanilla GD', markersize=4)
plt.plot(start[0], start[1], 'go', markersize=10)
plt.plot(0, 0, 'r*', markersize=15)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Vanilla GD on Ravine (oscillates!)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(122)
plt.contour(X, Y, Z, levels=30, alpha=0.5)
plt.plot(traj_momentum[:, 0], traj_momentum[:, 1], 'bo-', label='Momentum', markersize=4)
plt.plot(start[0], start[1], 'go', markersize=10)
plt.plot(0, 0, 'r*', markersize=15)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Momentum GD (smoother path!)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Vanilla GD final distance: {np.linalg.norm(traj_vanilla[-1]):.4f}")
print(f"Momentum GD final distance: {np.linalg.norm(traj_momentum[-1]):.4f}")

---
## Part 4: RMSprop

RMSprop: Adaptive learning rates per parameter.

**Your notes here:**


In [ ]:
def rmsprop(grad_func, start_pos, lr=0.1, decay=0.9, eps=1e-8, n_iters=50):
    """
    RMSprop: Root Mean Square Propagation.
    
    Adapts learning rate for each parameter based on recent gradients.
    
    v_t = β * v_{t-1} + (1-β) * (∇f)^2    (running avg of squared gradients)
    θ_t = θ_{t-1} - (lr / sqrt(v_t + ε)) * ∇f
    
    Intuition: If a parameter has large gradients, divide by large number (slower updates)
               If small gradients, divide by small number (faster updates)
    """
    pos = start_pos.copy()
    v = np.zeros_like(pos)  # running average of squared gradients
    trajectory = [pos.copy()]
    
    for i in range(n_iters):
        grad = grad_func(pos[0], pos[1])
        
        # Update running average of squared gradients
        v = decay * v + (1 - decay) * grad**2
        
        # Adaptive learning rate per parameter
        pos = pos - (lr / (np.sqrt(v) + eps)) * grad
        trajectory.append(pos.copy())
    
    return np.array(trajectory)

traj_rmsprop = rmsprop(ravine_grad, start, lr=0.5, decay=0.9, n_iters=50)

plt.figure(figsize=(8, 8))
plt.contour(X, Y, Z, levels=30, alpha=0.5)
plt.plot(traj_rmsprop[:, 0], traj_rmsprop[:, 1], 'mo-', label='RMSprop', markersize=4)
plt.plot(start[0], start[1], 'go', markersize=10)
plt.plot(0, 0, 'r*', markersize=15)
plt.xlabel('x')
plt.ylabel('y')
plt.title('RMSprop (adapts to geometry!)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"RMSprop final distance: {np.linalg.norm(traj_rmsprop[-1]):.4f}")

---
## Part 5: Adam (The King)

Adam = Momentum + RMSprop. Default optimizer for most problems.

**Your notes here:**


In [ ]:
def adam(grad_func, start_pos, lr=0.1, beta1=0.9, beta2=0.999, eps=1e-8, n_iters=50):
    """
    Adam: Adaptive Moment Estimation.
    
    Combines:
    - Momentum (first moment of gradients)
    - RMSprop (second moment of gradients)
    
    m_t = β1 * m_{t-1} + (1-β1) * ∇f        (momentum)
    v_t = β2 * v_{t-1} + (1-β2) * (∇f)^2    (RMSprop)
    
    # Bias correction (important for early iterations)
    m_hat = m_t / (1 - β1^t)
    v_hat = v_t / (1 - β2^t)
    
    θ_t = θ_{t-1} - (lr / sqrt(v_hat) + ε) * m_hat
    
    Default values: β1=0.9, β2=0.999, lr=0.001
    """
    pos = start_pos.copy()
    m = np.zeros_like(pos)  # first moment (momentum)
    v = np.zeros_like(pos)  # second moment (RMSprop)
    trajectory = [pos.copy()]
    
    for t in range(1, n_iters + 1):
        grad = grad_func(pos[0], pos[1])
        
        # Update biased first moment
        m = beta1 * m + (1 - beta1) * grad
        
        # Update biased second moment
        v = beta2 * v + (1 - beta2) * grad**2
        
        # Bias correction
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)
        
        # Update parameters
        pos = pos - (lr / (np.sqrt(v_hat) + eps)) * m_hat
        trajectory.append(pos.copy())
    
    return np.array(trajectory)

traj_adam = adam(ravine_grad, start, lr=0.5, n_iters=50)

plt.figure(figsize=(8, 8))
plt.contour(X, Y, Z, levels=30, alpha=0.5)
plt.plot(traj_adam[:, 0], traj_adam[:, 1], 'co-', label='Adam', markersize=4)
plt.plot(start[0], start[1], 'go', markersize=10)
plt.plot(0, 0, 'r*', markersize=15)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Adam (smooth + adaptive!)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Adam final distance: {np.linalg.norm(traj_adam[-1]):.4f}")

### Compare All Optimizers

**Your notes here:**


In [ ]:
# Run all optimizers
start = np.array([8.0, 3.0])

optimizers = [
    ('Vanilla GD', vanilla_gradient_descent(ravine_grad, start, lr=0.05, n_iters=50), 'red'),
    ('Momentum', momentum_gd(ravine_grad, start, lr=0.05, momentum=0.9, n_iters=50), 'blue'),
    ('RMSprop', rmsprop(ravine_grad, start, lr=0.5, n_iters=50), 'magenta'),
    ('Adam', adam(ravine_grad, start, lr=0.5, n_iters=50), 'cyan'),
]

plt.figure(figsize=(12, 10))
plt.contour(X, Y, Z, levels=40, alpha=0.3)

for name, traj, color in optimizers:
    plt.plot(traj[:, 0], traj[:, 1], 'o-', color=color, label=name, 
             markersize=3, linewidth=2, alpha=0.7)

plt.plot(start[0], start[1], 'go', markersize=15, label='Start', zorder=5)
plt.plot(0, 0, 'r*', markersize=20, label='Global min', zorder=5)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Optimizer Comparison on Ravine Function')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nFinal distances from minimum:")
for name, traj, _ in optimizers:
    dist = np.linalg.norm(traj[-1])
    print(f"{name:15s}: {dist:.6f}")

---
## Part 6: Learning Rate Schedules

Adjusting learning rate during training.

**Your notes here:**


In [ ]:
def constant_lr(initial_lr, epoch, total_epochs):
    """Constant learning rate"""
    return initial_lr

def step_decay(initial_lr, epoch, total_epochs, drop=0.5, epochs_drop=10):
    """Drop by factor every N epochs"""
    return initial_lr * (drop ** (epoch // epochs_drop))

def exponential_decay(initial_lr, epoch, total_epochs, decay_rate=0.95):
    """Exponential decay"""
    return initial_lr * (decay_rate ** epoch)

def cosine_annealing(initial_lr, epoch, total_epochs):
    """
    Cosine annealing (popular in transformers).
    Smoothly decreases from initial_lr to 0.
    """
    return initial_lr * 0.5 * (1 + np.cos(np.pi * epoch / total_epochs))

def warmup_cosine(initial_lr, epoch, total_epochs, warmup_epochs=10):
    """
    Linear warmup + cosine decay (used in GPT, BERT).
    
    Why warmup? Large learning rates at start can destabilize training.
    Warmup gradually increases LR from 0 to initial_lr.
    """
    if epoch < warmup_epochs:
        # Linear warmup
        return initial_lr * (epoch / warmup_epochs)
    else:
        # Cosine decay
        progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
        return initial_lr * 0.5 * (1 + np.cos(np.pi * progress))

# Visualize schedules
total_epochs = 100
initial_lr = 0.1
epochs = np.arange(total_epochs)

schedules = [
    ('Constant', [constant_lr(initial_lr, e, total_epochs) for e in epochs]),
    ('Step Decay', [step_decay(initial_lr, e, total_epochs) for e in epochs]),
    ('Exponential', [exponential_decay(initial_lr, e, total_epochs) for e in epochs]),
    ('Cosine', [cosine_annealing(initial_lr, e, total_epochs) for e in epochs]),
    ('Warmup+Cosine', [warmup_cosine(initial_lr, e, total_epochs) for e in epochs]),
]

plt.figure(figsize=(12, 6))
for name, schedule in schedules:
    plt.plot(epochs, schedule, label=name, linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedules')
plt.legend()
plt.grid(True)
plt.show()

print("Warmup + Cosine is standard for transformers (GPT, BERT, etc.)")

---
## Summary: Which Optimizer to Use?

| Optimizer | When to Use | Typical LR |
|-----------|-------------|------------|
| SGD | Simple problems, convex optimization | 0.01-0.1 |
| SGD + Momentum | Computer vision (CNNs), when you need stability | 0.01-0.1 |
| RMSprop | RNNs, non-stationary problems | 0.001 |
| Adam | **Default choice**, transformers, most DL | 0.0001-0.001 |
| AdamW | Transformers (better weight decay) | 0.0001-0.001 |

**Rule of thumb**: Start with Adam (lr=0.001). If training is unstable, add warmup.

**Your notes here:**


---
## Key Takeaways

1. **Gradient descent**: Follow negative gradient to minimize loss
2. **Batch size matters**: Full batch (stable), SGD (noisy), mini-batch (practical)
3. **Momentum**: Accelerates convergence, smooths path
4. **Adaptive LR** (RMSprop/Adam): Different learning rates per parameter
5. **Adam**: Best all-around optimizer (momentum + adaptive LR)
6. **LR schedules**: Warmup prevents instability, cosine decay improves final performance

**Your notes here:**
